In [ ]:
import pytesseract
import cv2
import PyPDF2
import re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

In [ ]:
# Get all files recursively
top_dir = "/Users/timvigers/Desktop/G4 from Epic Rotated/"
files_only = []
for path, subdirs, files in os.walk(top_dir):
    for name in files:
        if ".pdf" in name:
            files_only.append(str(os.path.join(path, name)))

In [ ]:
# Define function for image reading with and without thresholding
def get_text(img,threshold = True):
    if threshold == True:
        gry = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
        flt = cv2.adaptiveThreshold(gry,300, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 15, 16)
        text = pytesseract.image_to_string(flt)
    else:
        text = pytesseract.image_to_string(img)
    text = re.sub("[=!?',;.\n|+ ]",'',text)
    return(text)
# Function for searching both texts with regex
def regex(text,text2,expression):
    var = re.search(expression, text)
    if type(var) != re.Match:# Check missing and re-try without image thresholding
        var = re.search(expression, text2)    
    var = re.findall('\d+', var[0])   
    return(var)

In [ ]:
results={'file': [], 'avg_glucose': [], 'sens_use': [], 'calib': [], 'sd': [],'high': [], 'target': [], 'low': []}
for file in files_only:
    results['file'].append(re.sub('/Users/timvigers/Desktop/G4 from Epic Rotated/','',file))
    # List PDF pages
    pages = convert_from_path(file)
    # Page 1
    t1 = get_text(pages[0])
    t2 = get_text(pages[0],threshold=False)
    # Check document type
    if 'EstimatedA1C' not in t1:
        # Pull all variables
        # Glucose
        avg_glucose = regex(t1,t2,'AverageGlucose\d*')   
        results['avg_glucose'].append(''.join(avg_glucose))
        # Sensor use
        sens_use = regex(t1,t2,'\d{1,2}of14Days')
        results['sens_use'].append(' of '.join(sens_use))
        # Calibrations
        calib = regex(t1,t2,'Calibrations/day\d*')
        results['calib'].append(''.join(calib))
        # SD
        sd = regex(t1,t2,'StandardDeviation\d*')   
        results['sd'].append(''.join(sd))
        # Percent high
        high = regex(t1,t2,'\d*%High')
        results['high'].append(''.join(high))
        # Percent target
        target = regex(t1,t2,'\d*%Target')
        results['target'].append(''.join(target))
        # Percent low
        low = regex(t1,t2,'\d*%Low')
        results['low'].append(''.join(low))
    elif 'EstimatedA1C' not in t1:
results